In [15]:
from ssl import CERT_NONE
from pprint import pprint

from nostr.event import Event
from nostr.key import PrivateKey
from nostr.relay_manager import RelayManager

In [12]:
# Setup Alice's keys
privkey: PrivateKey = PrivateKey().from_nsec(PRIVKEY)
print(privkey.public_key.bech32())

npub1tqh0wyekvvhfc82ux52gd4tahusvfx7wt2789vtvsny3jed5x0gqht6tun


In [17]:
# Connect to relays
relay_manager = RelayManager()
relay_manager.add_relay(RELAY)
relay_manager.add_relay("wss://relay.damus.io")
relay_manager.add_relay("wss://relay.snort.social")
relay_manager.add_relay("wss://nos.lol")
relay_manager.open_connections({"cert_reqs": CERT_NONE})

pprint(relay_manager.relays)

{'wss://nor.st': <nostr.relay.Relay object at 0x1197c8310>,
 'wss://nos.lol': <nostr.relay.Relay object at 0x1197e3690>,
 'wss://relay.damus.io': <nostr.relay.Relay object at 0x11a2d0e10>,
 'wss://relay.snort.social': <nostr.relay.Relay object at 0x11a2d3cd0>}


In [7]:
RELAY: str = 'wss://nor.st'
PRIVKEY: str = 'nsec1mp95lpaxpd50dvvh7cq0azasvdl8cd0qgu55qtlmz3he25wj2t2q52lx9k'